In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

from google.colab import drive
drive.mount('/content/drive')



Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Mounted at /content/drive


In [3]:
import torch.optim as optim
criterion = torch.nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

def accuracy(out, labels):
  _,pred = torch.max(out, dim=1)
  return torch.sum(pred==labels).item()


num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)

In [4]:
from torchvision.transforms.transforms import Normalize
import numpy as np
import pandas as pd
import imageio 
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.preprocessing import StandardScaler
from torch.nn.functional import normalize

scaler = StandardScaler() 



preprocess=transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class MyDataset(Dataset):


    def __init__(self, csv_file, root_dir, transform=preprocess):

        self.brix= pd.read_csv(csv_file,sep=",")
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.brix)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
  
        l=self.brix.iloc[idx,4].split(".")
        name=l[0]+"_wb.tiff"
        img_name = os.path.join(self.root_dir,
                                name)


        image = imageio.imread(img_name)
        label=np.float32(self.brix.iloc[idx,6])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample['image']=np.float32(sample['image'])
            sample['image'] = self.transform(sample['image'])
            sample['image'] = normalize(sample['image'], p=1.0, dim = 0)
            

        return sample

brix_dataset = MyDataset(csv_file=open('/content/drive/MyDrive/brix_labels.csv', 'r'),
                                    root_dir='drive/MyDrive/white_balanced_images')

input_image=brix_dataset[2]['image']
print(brix_dataset[2]['label'])
input_tensor=input_image



input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model


if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(input_batch)
print(probabilities)

1.0
tensor([[[[0.5293, 0.3927, 0.4770,  ..., 0.3795, 0.3667, 0.3390],
          [0.3611, 0.4118, 0.3611,  ..., 0.3302, 0.2946, 0.3859],
          [0.4869, 0.4853, 0.3337,  ..., 0.3320, 0.4028, 0.3557],
          ...,
          [0.4452, 0.4314, 0.4087,  ..., 0.4188, 0.4036, 0.3609],
          [0.3965, 0.3963, 0.4229,  ..., 0.4180, 0.4079, 0.3877],
          [0.4175, 0.3728, 0.4175,  ..., 0.3861, 0.4400, 0.4144]],

         [[0.2961, 0.4258, 0.2873,  ..., 0.3800, 0.4377, 0.4184],
          [0.3261, 0.2481, 0.3261,  ..., 0.4056, 0.3818, 0.4001],
          [0.2724, 0.3547, 0.3773,  ..., 0.4413, 0.3731, 0.4387],
          ...,
          [0.2881, 0.3444, 0.3079,  ..., 0.2786, 0.3300, 0.3262],
          [0.3286, 0.2985, 0.2921,  ..., 0.2822, 0.3564, 0.3211],
          [0.2973, 0.3257, 0.2973,  ..., 0.3221, 0.3340, 0.2862]],

         [[0.1745, 0.1814, 0.2357,  ..., 0.2405, 0.1956, 0.2426],
          [0.3128, 0.3401, 0.3128,  ..., 0.2642, 0.3236, 0.2140],
          [0.2407, 0.1600, 0.2890,  ..

In [13]:
n_epochs = 5
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
dataset_size=MyDataset.__len__(brix_dataset)

for epoch in range(1, n_epochs+1):
  running_loss = 0.0
  correct = 0
  total=0
  print(f'Epoch {epoch}\n')
  
  for batch_idx in range(dataset_size):
     sample=brix_dataset[batch_idx]
     data_=sample['image']
     target_=torch.tensor(sample['label']).unsqueeze(0)
     if torch.cuda.is_available():
      data_, target_ = data_.to('cuda'), target_.to('cuda')
     optimizer.zero_grad()
     data_ = data_.unsqueeze(0)      
     outputs = model(data_).view(1)
     #print(outputs.size())
     probabilities = torch.nn.functional.softmax(outputs)
     #print(target_,"and",probabilities)
     loss = criterion(probabilities, target_)
     loss.backward()
     optimizer.step()
     running_loss += loss.item()
     #_,pred = torch.max(outputs, dim=1)
     if probabilities[0]>0.5:
       pred=torch.tensor(np.float32([1]))
     else:
        pred=torch.tensor(np.float32([0]))
     pred=pred.to('cuda')
     correct += torch.sum(pred==target_).item()
     total += target_.size(0)
     if (batch_idx) % 20 == 0:
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        .format(epoch, n_epochs, batch_idx, dataset_size, loss.item()))
  train_acc.append(100 * correct / total)
  train_loss.append(running_loss/dataset_size)
  print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
  batch_loss = 0  
  total_t=0
  correct_t=0

Epoch 1



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/5], Step [0/150], Loss: 0.0000
Epoch [1/5], Step [20/150], Loss: 0.0000
Epoch [1/5], Step [40/150], Loss: 0.0000
Epoch [1/5], Step [60/150], Loss: 0.0000
Epoch [1/5], Step [80/150], Loss: 0.0000
Epoch [1/5], Step [100/150], Loss: 0.0000
Epoch [1/5], Step [120/150], Loss: 0.0000
Epoch [1/5], Step [140/150], Loss: 100.0000

train-loss: 27.3333, train-acc: 72.6667
Epoch 2

Epoch [2/5], Step [0/150], Loss: 0.0000
Epoch [2/5], Step [20/150], Loss: 0.0000
Epoch [2/5], Step [40/150], Loss: 0.0000
Epoch [2/5], Step [60/150], Loss: 0.0000
Epoch [2/5], Step [80/150], Loss: 0.0000
Epoch [2/5], Step [100/150], Loss: 0.0000
Epoch [2/5], Step [120/150], Loss: 0.0000
Epoch [2/5], Step [140/150], Loss: 100.0000

train-loss: 27.3333, train-acc: 72.6667
Epoch 3

Epoch [3/5], Step [0/150], Loss: 0.0000
Epoch [3/5], Step [20/150], Loss: 0.0000
Epoch [3/5], Step [40/150], Loss: 0.0000
Epoch [3/5], Step [60/150], Loss: 0.0000
Epoch [3/5], Step [80/150], Loss: 0.0000
Epoch [3/5], Step [100/150], Los